<a href="https://colab.research.google.com/github/debugk/NER_Covid19/blob/AddEval/NER_bert_bilstm_crf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/gdrive')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
#install the amazing transformers package by huggingface
!pip install transformers
import transformers
from transformers import BertTokenizer, BertConfig

     |████████████████████████████████| 778kB 2.7MB/s 
     |████████████████████████████████| 1.1MB 15.2MB/s 
     |████████████████████████████████| 890kB 17.0MB/s 
     |████████████████████████████████| 3.0MB 13.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=3c84a150967e754ac79a22a15e1108fbb3b90945e3f21c0e5624e3141fe41c81
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

tf_model = TFBertModel.from_pretrained('bert-base-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
import nltk
nltk.download('punkt')

# Word embedding

In [21]:

#==========================================================================================
def prepare_tokentags(data, max_length = 150, fdebug=False):
    data_tags   = []
    data_tokens = []

    sentences = nltk.tokenize.sent_tokenize(data['text'])
    entities  = sorted(data['entities'], key=lambda entry: entry['start'], reverse=True) # sort by the entry start point, reverse

    # The target is the group of the words in top entry, i.e. papillomavirus -> 
    entry = entities.pop()
    target_tokens = tokenizer.tokenize(entry['entity'])
    target_token  = '_'.join(target_tokens)  
    len_targets   = len(target_tokens)
    
    is_never_match = True

    for sentence in sentences:
        # init the sentence tags
        sentence_tags = ['O']*max_length
        # Add the special tokens.
        sentence = "[CLS] " + sentence + "[SEP]" 

        tokens = tokenizer.tokenize(sentence)

        if len(tokens) < max_length:
            tokens += ["[PAD]"]*(max_length - len(tokens))
        
        i = 0

        while i < min(len(tokens), max_length): # some len(tokens) > 150 
            end_point = i+len_targets if (i+len_targets) < len(tokens) else len(tokens)
            match_token = '_'.join(tokens[i:i+len_targets])

            if match_token ==  target_token:
                is_never_match = False

                sentence_tags[i] = 'B-' + entry['type']

                for j in range(1, len_targets):
                    if i+j >= max_length: continue 

                    sentence_tags[i+j] = 'I-' + entry['type']

                # update the position
                i += len_targets 

                if entities:
                    # remove the top entry, move to next entry
                    entry = entities.pop()
                    target_tokens = tokenizer.tokenize(entry['entity'])  
                    target_token  = '_'.join(target_tokens)  
                    len_targets   = len(target_tokens) 
                else:
                    # entities is empty, stop loop
                    i = len(tokens)
            else:
                  i += 1

        if fdebug:
            for token, tag in zip(tokens, sentence_tags):
                print("{}\t{}".format(tag, token))
        
        data_tokens += [tokens]
        data_tags   += [sentence_tags]           

    if entities: 
        print(sentences)
        print("unmatched tags: ", entities)
    elif is_never_match:
        print(sentences)
        print("Only one tag and it is not matched: ", entry)

    return (data_tokens, data_tags)


#==========================================================================================
def prepare_embedding(datas, max_length = 150, fdebug=False):
    sentences = []
    
    for data in datas:
        sentences += nltk.tokenize.sent_tokenize(data['text'])
    
    # it will extrat the same token as "tokenizer.encode(sentence)" or "tokenizer.tokenize(sentence); tokenizer.convert_tokens_to_ids()"
    tf_inputs = tokenizer(sentences, padding=True, pad_to_multiple_of=max_length, return_attention_mask=True, return_tensors="tf") 

    if fdebug:
        print(tf_inputs)

    tf_outputs = tf_model(tf_inputs)

    # use the second-to-last layer for embedding.   
    last_hidden_states = tf_outputs[0]

    return last_hidden_states # tensorflow tensor

# Word embedding test

In [22]:
test_data = {"text": "Progress towards development of an HIV vaccine: report of the AIDS Vaccine 2009 Conference\tThe search for an HIV/AIDS vaccine is steadily moving ahead, generating and validating new concepts in terms of novel vectors for antigen delivery and presentation, new vaccine and adjuvant strategies, alternative approaches to design HIV-1 antigens for eliciting protective cross-neutralising antibodies, and identification of key mechanisms in HIV infection and modulation of the immune system. All these different perspectives are contributing to the unprecedented challenge of developing a protective HIV-1 vaccine.", 
             "entities": [
                          #{"entity": "HIV vaccine", "type": "Drug", "start": 35, "end": 46}, 
                          #{"entity": "AIDS", "type": "Disease", "start": 62, "end": 66}, 
                          #{"entity": "HIV/AIDS vaccine", "type": "Drug", "start": 109, "end": 125}, 
                          {"entity": "vaccine", "type": "Drug", "start": 118, "end": 125}, 
                          {"entity": "vaccine", "type": "Drug", "start": 260, "end": 267}, 
                          {"entity": "HIV-1 antigens", "type": "Virus", "start": 326, "end": 340}, 
                          {"entity": "cross-neutralising antibodies", "type": "Chemical", "start": 366, "end": 395}, 
                          {"entity": "HIV infection", "type": "Phenotype", "start": 437, "end": 450}, 
                          {"entity": "HIV-1 vaccine", "type": "Drug", "start": 596, "end": 609}]}


test_tokens, test_tags = prepare_tokentags(test_data, fdebug=True) 

O	[CLS]
O	Progress
O	towards
O	development
O	of
O	an
O	HIV
B-Drug	vaccine
O	:
O	report
O	of
O	the
O	AIDS
O	V
O	##ac
O	##cine
O	2009
O	Conference
O	The
O	search
O	for
O	an
O	HIV
O	/
O	AIDS
B-Drug	vaccine
O	is
O	steadily
O	moving
O	ahead
O	,
O	generating
O	and
O	valid
O	##ating
O	new
O	concepts
O	in
O	terms
O	of
O	novel
O	vectors
O	for
O	anti
O	##gen
O	delivery
O	and
O	presentation
O	,
O	new
O	vaccine
O	and
O	ad
O	##ju
O	##vant
O	strategies
O	,
O	alternative
O	approaches
O	to
O	design
B-Virus	HIV
I-Virus	-
I-Virus	1
I-Virus	anti
I-Virus	##gens
O	for
O	el
O	##icit
O	##ing
O	protective
B-Chemical	cross
I-Chemical	-
I-Chemical	neutral
I-Chemical	##ising
I-Chemical	antibodies
O	,
O	and
O	identification
O	of
O	key
O	mechanisms
O	in
B-Phenotype	HIV
I-Phenotype	infection
O	and
O	m
O	##od
O	##ulation
O	of
O	the
O	immune
O	system
O	.
O	[SEP]
O	[PAD]
O	[PAD]
O	[PAD]
O	[PAD]
O	[PAD]
O	[PAD]
O	[PAD]
O	[PAD]
O	[PAD]
O	[PAD]
O	[PAD]
O	[PAD]
O	[PAD]
O	[PAD]
O	[PAD]
O	[PAD]
O	[PAD]
O	[PAD]
O	[PAD]
O	[PA

In [23]:
tokenizer.tokenize('HIV-1 antigens')

['HIV', '-', '1', 'anti', '##gens']

In [24]:
test_embedding = prepare_embedding([test_data], fdebug=True)

{'input_ids': <tf.Tensor: shape=(2, 150), dtype=int32, numpy=
array([[  101, 12798,  2019,  1718,  1104,  1126,  9622, 20034,   131,
         2592,  1104,  1103,  9837,   159,  7409, 15459,  1371,  3047,
         1109,  3403,  1111,  1126,  9622,   120,  9837, 20034,  1110,
        11383,  2232,  3075,   117, 12713,  1105,  9221,  3798,  1207,
         8550,  1107,  2538,  1104,  2281, 21118,  1111,  2848,  4915,
         6779,  1105,  8685,   117,  1207, 20034,  1105,  8050,  9380,
        14194, 10700,   117,  4174,  8015,  1106,  1902,  9622,   118,
          122,  2848, 21144,  1111,  8468, 22308,  1158,  9760,  2771,
          118,  8795,  7131, 26491,   117,  1105,  9117,  1104,  2501,
        10748,  1107,  9622,  8974,  1105,   182,  5412,  6856,  1104,
         1103, 11650,  1449,   119,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0, 

# Read training file

In [10]:
import json
input_dir = "/content/gdrive/My Drive/Covid19_data/"

f = open(input_dir+'new_train.json','r')
datas = []
for line in f:
  datas.append(json.loads(line))

In [25]:
train_tags = []

for data in datas:
    _, cur_tags = prepare_tokentags(data)
    
    train_tags += cur_tags

['Safety and efficacy of intravenous bimagrumab in inclusion body myositis (RESILIENT): a randomised, double-blind, placebo-controlled phase 2b trial\tBimagrumab showed a good safety profile, relative to placebo, in individuals with inclusion body myositis but did not improve 6MWD.', 'The strengths of our study are that, to the best of our knowledge, it is the largest randomised controlled trial done in people with inclusion body myositis, and it provides important natural history data over 12 months.']
unmatched tags:  [{'entity': 'inclusion body myositis', 'type': 'Disease', 'start': 413, 'end': 436}, {'entity': '6MWD', 'type': 'Gene', 'start': 274, 'end': 278}]
['Progress towards development of an HIV vaccine: report of the AIDS Vaccine 2009 Conference\tThe search for an HIV/AIDS vaccine is steadily moving ahead, generating and validating new concepts in terms of novel vectors for antigen delivery and presentation, new vaccine and adjuvant strategies, alternative approaches to desig

In [ ]:
train_inputs_embeds = prepare_embedding(datas)